# Financial Data Analysis with AI

This notebook demonstrates how to analyze financial transaction data using DuckDB and AI. The data is stored in a CSV file within the data folder and we'll explore various analytical insights.

## Import Required Libraries

Import pandas, numpy, and other necessary libraries for data analysis.

In [ ]:
import pandas as pd
import numpy as np
import duckdb
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## Set Up Data Directory Path

Define the path to the data folder and verify it exists.

In [ ]:
# Define data directory path
data_dir = Path("/app/data")  # Docker container path
csv_file = data_dir / "sample_data.csv"

# For local development, you might use:
# data_dir = Path("../data")
# csv_file = data_dir / "sample_data.csv"

print(f"Data directory: {data_dir}")
print(f"CSV file path: {csv_file}")
print(f"Data directory exists: {data_dir.exists()}")
print(f"CSV file exists: {csv_file.exists()}")

## Load CSV Data from Data Folder

Use pandas to read the sample_data.csv file from the data directory.

In [ ]:
# Load the CSV data
try:
    df = pd.read_csv(csv_file)
    print(f"Successfully loaded {len(df)} transactions from {csv_file}")
    print(f"Data shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: File not found at {csv_file}")
    # Fallback for local development
    try:
        df = pd.read_csv("../data/sample_data.csv")
        print("Loaded data using fallback path")
    except:
        print("Could not load data from any path")
        df = None

## Display Data Information

Show the first few rows, data types, and basic information about the dataset.

In [ ]:
if df is not None:
    print("=== FIRST 5 ROWS ===")
    display(df.head())
    
    print("\n=== DATA TYPES ===")
    display(df.dtypes)
    
    print("\n=== DATASET INFO ===")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")
    
    print("\n=== MISSING VALUES ===")
    display(df.isnull().sum())

## Basic Data Exploration

Perform initial exploration including summary statistics and data shape analysis.

In [ ]:
if df is not None:
    # Convert date column to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    print("=== SUMMARY STATISTICS ===")
    display(df.describe())
    
    print("\n=== TRANSACTION CATEGORIES ===")
    category_counts = df['category'].value_counts()
    display(category_counts)
    
    print("\n=== ACCOUNT TYPES ===")
    account_counts = df['account_type'].value_counts()
    display(account_counts)
    
    print("\n=== FINANCIAL SUMMARY ===")
    total_income = df[df['amount'] > 0]['amount'].sum()
    total_expenses = df[df['amount'] < 0]['amount'].sum()
    net_amount = df['amount'].sum()
    
    print(f"Total Income: ${total_income:,.2f}")
    print(f"Total Expenses: ${abs(total_expenses):,.2f}")
    print(f"Net Amount: ${net_amount:,.2f}")
    print(f"Date Range: {df['date'].min()} to {df['date'].max()}")

## Monitor AI Model Download Progress

Check the status of the Llama 3.1 model download and test AI functionality once ready.

In [21]:
import requests
import subprocess
import time
import json

# Check if Ollama model is ready
def check_ollama_status():
    try:
        # Try to list models using docker-compose exec
        result = subprocess.run(['docker-compose', 'exec', 'ollama', 'ollama', 'list'], 
                              capture_output=True, text=True)
        
        if result.returncode == 0:
            lines = result.stdout.strip().split('\n')
            if len(lines) > 1:  # Header + at least one model
                print("✅ Ollama models available:")
                print(result.stdout)
                return True
            else:
                print("⏳ No models available yet - still downloading...")
                return False
        else:
            print(f"❌ Error checking models: {result.stderr}")
            return False
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

# Test AI functionality
def test_ai_query():
    try:
        response = requests.post('http://localhost:5000/ask', 
                               json={'question': 'What is the total spending amount?'})
        if response.status_code == 200:
            result = response.json()
            print("✅ AI Query successful!")
            print(f"Question: What is the total spending amount?")
            print(f"SQL: {result.get('sql_query', 'N/A')}")
            print(f"Answer: {result.get('ai_response', 'N/A')}")
            return True
        else:
            print(f"❌ AI Query failed: {response.status_code}")
            print(response.text)
            return False
    except Exception as e:
        print(f"❌ Error testing AI: {e}")
        return False

# Monitor download progress
def monitor_download():
    print("🔍 Checking Ollama model status...")
    
    if check_ollama_status():
        print("\n🚀 Testing AI functionality...")
        test_ai_query()
    else:
        print("\n📊 To check download progress, run in terminal:")
        print("docker-compose logs ollama --tail 10")
        
        print("\n⏰ Download should complete in ~25 seconds at current speed")
        print("You can re-run this cell to check again!")

# Run the monitoring
monitor_download()

🔍 Checking Ollama model status...
✅ Ollama models available:
NAME               ID              SIZE      MODIFIED       
llama3.1:latest    46e0c10c039e    4.9 GB    30 minutes ago    


🚀 Testing AI functionality...
✅ Ollama models available:
NAME               ID              SIZE      MODIFIED       
llama3.1:latest    46e0c10c039e    4.9 GB    30 minutes ago    


🚀 Testing AI functionality...
✅ AI Query successful!
Question: What is the total spending amount?
SQL: SELECT SUM(CASE WHEN amount < 0 THEN -amount ELSE 0 END) AS total_spending_amount
FROM transactions;
Answer: The total spending amount is $1,363.15. This result indicates that there have been negative transactions totaling this amount in the database, as evidenced by the fact that some transaction amounts are less than 0. The sum of these absolute values provides a measure of overall spending activity within the system.
✅ AI Query successful!
Question: What is the total spending amount?
SQL: SELECT SUM(CASE WHEN amount 

: 

# 🤖 AI-Powered Financial Analysis

Now let's use the Ollama AI service to get intelligent insights about our financial data. We'll connect directly to the Ollama service running in Docker.

In [20]:
# Install required packages for AI communication
import requests
import json
import pandas as pd

# Ollama service configuration - using localhost since we're in simplified stack
OLLAMA_URL = "http://localhost:11434"  # Localhost access to exposed port
OLLAMA_API = f"{OLLAMA_URL}/api/generate"

print("✅ AI libraries imported successfully!")
print(f"🔗 Ollama API URL: {OLLAMA_API}")

# Test connection to Ollama
try:
    response = requests.get(f"{OLLAMA_URL}/api/tags", timeout=5)
    if response.status_code == 200:
        models = response.json()
        print(f"🤖 Available models: {[model['name'] for model in models.get('models', [])]}")
    else:
        print("⚠️ Could not fetch models list")
except Exception as e:
    print(f"❌ Connection error: {e}")
    print("💡 Make sure Ollama service is running and llama3.1 model is downloaded")

✅ AI libraries imported successfully!
🔗 Ollama API URL: http://localhost:11434/api/generate
🤖 Available models: []


In [32]:
import requests
import json
import subprocess

def ask_ai(question, context_data=None, use_data=False):
    """
    Ask the AI a question about financial data
    
    Args:
        question (str): The question to ask
        context_data (str): Optional data context to include
        use_data (bool): If True, uses Flask API for data queries. If False, uses direct AI for general advice
    
    Returns:
        str: AI response
    """
    
    # For general financial advice, use direct AI without database queries
    if not use_data and "my spending" not in question.lower() and "my data" not in question.lower():
        try:
            # Use docker-compose exec for general questions
            prompt = f"""You are a helpful financial advisor. Answer this question with general financial advice:

{question}

Provide practical, actionable advice without requiring specific financial data."""
            
            result = subprocess.run(
                ['docker-compose', 'exec', '-T', 'ollama', 'sh', '-c', f'echo "{prompt}" | ollama run llama3.1:latest'],
                capture_output=True,
                text=True,
                timeout=60,
                cwd='E:\\Workspace\\td25-money'
            )
            
            if result.returncode == 0 and result.stdout.strip():
                return result.stdout.strip()
            else:
                # Fallback to Flask API
                return ask_ai_with_data(question, context_data)
                
        except Exception as e:
            # Fallback to Flask API
            return ask_ai_with_data(question, context_data)
    else:
        # For data-specific questions, use Flask API
        return ask_ai_with_data(question, context_data)

def ask_ai_with_data(question, context_data=None):
    """Helper function to use Flask API for data-specific questions"""
    # Add context to the question if provided
    if context_data:
        enhanced_question = f"{question}\n\nContext: {context_data}"
    else:
        enhanced_question = question
    
    try:
        # Use the Flask API for data-related questions
        response = requests.post(
            "http://localhost:5000/ask",
            json={"question": enhanced_question},
            timeout=60
        )
        
        if response.status_code == 200:
            result = response.json()
            return result.get('ai_response', 'No AI response received')
        else:
            return f"❌ Flask API Error {response.status_code}: {response.text}"
            
    except requests.exceptions.Timeout:
        return "⏰ AI response timed out (60 seconds)"
    except requests.exceptions.ConnectionError:
        return "❌ Connection error: Could not connect to Flask API"
    except Exception as e:
        return f"❌ Error calling AI: {str(e)}"

print("✅ AI helper function updated with smart routing!")
print("💡 Use ask_ai('your question') for general advice")
print("💡 Use ask_ai('your question', use_data=True) for data analysis")

✅ AI helper function updated with smart routing!
💡 Use ask_ai('your question') for general advice
💡 Use ask_ai('your question', use_data=True) for data analysis


In [33]:
# Example 1: Ask AI for general financial advice
print("🤖 Asking AI for budgeting advice...")
response = ask_ai("What are the best practices for creating a monthly budget?")
print(f"AI Response:\n{response}")
print("\n" + "="*50 + "\n")

🤖 Asking AI for budgeting advice...


Exception in thread Thread-17 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\lzhou\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\Users\lzhou\AppData\Roaming\Python\Python313\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "c:\Users\lzhou\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 992, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lzhou\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 1611, in _readerthread
    buffer.append(fh.read())
                  ~~~~~~~^^
UnicodeDecodeError: 'gbk' codec can't decode byte 0x99 in position 223: illegal multibyte sequence


AI Response:
Creating a monthly budget! A crucial step towards taking control of your finances. I'm happy to share some best practices to help you create a effective and sustainable budget.

**1. Track Your Income**: Start by calculating how much money comes into your household each month from all sources, including salary, investments, and any side hustles. Be sure to account for irregular income, like bonuses or freelance work.

**2. Categorize Expenses**: Divide your expenses into categories, such as:
	* Housing (rent/mortgage, utilities)
	* Transportation (car loan/gas/insurance)
	* Food (groceries/dining out)
	* Insurance (health/life/disability)
	* Debt repayment (credit cards/loans)
	* Entertainment (hobbies/travel)
	* Savings
	* Miscellaneous (gifts/subscriptions)

**3. Set Financial Goals**: Identify what you want to achieve with your budget, such as:
	* Saving for a specific goal (e.g., emergency fund, down payment on a house)
	* Paying off debt
	* Building an investment port

In [16]:
# Test data retrieval from the DuckDB API
import requests

# Test API connection
api_url = "http://localhost:5000"
print(f"Connecting to DuckDB API at: {api_url}")

# Check API health
health_response = requests.get(f"{api_url}/health")
print(f"API Health: {health_response.json()}")

# Get financial summary
summary_response = requests.get(f"{api_url}/summary")
summary_data = summary_response.json()
print(f"Financial Summary from API:")
print(f"  success: {summary_data.get('success')}")
print(f"  summary: {summary_data.get('summary')}")

# Get categories data
categories_response = requests.get(f"{api_url}/categories")
categories_data = categories_response.json()
print(f"\nCategories Response:")
print(f"  success: {categories_data.get('success')}")

# Access categories data using the correct key
if categories_data.get('success') and categories_data.get('categories'):
    categories = categories_data['categories']
    print(f"\nSpending by Category:")
    
    for category in categories:
        cat_name = category.get('category', 'Unknown')
        total_spent = category.get('total_spent', 0)
        total_earned = category.get('total_earned', 0)
        transaction_count = category.get('transaction_count', 0)
        
        if total_spent > 0:
            print(f"  {cat_name}: ${total_spent:.2f} spent ({transaction_count} transactions)")
        elif total_earned > 0:
            print(f"  {cat_name}: ${total_earned:.2f} earned ({transaction_count} transactions)")
else:
    print("  Error retrieving categories data")

print(f"\nData ready for AI analysis!")

Connecting to DuckDB API at: http://localhost:5000
API Health: {'service': 'duckdb-ai-service', 'status': 'healthy'}
Financial Summary from API:
  success: True
  summary: {'date_range': '2025-01-15 to 2025-02-03', 'net_amount': 1711.85, 'total_expenses': 1363.1499999999999, 'total_income': 3075.0, 'total_transactions': 20, 'unique_categories': 14}

Categories Response:
  success: True

Spending by Category:
  shopping: $450.00 spent (2 transactions)
  food: $222.67 spent (4 transactions)
  rent: $200.00 spent (1 transactions)
  insurance: $120.00 spent (1 transactions)
  fuel: $95.50 spent (1 transactions)
  utilities: $89.99 spent (1 transactions)
  entertainment: $65.30 spent (1 transactions)
  coffee: $40.50 spent (2 transactions)
  health: $35.20 spent (1 transactions)
  transportation: $25.00 spent (1 transactions)
  subscriptions: $18.99 spent (1 transactions)
  salary: $2500.00 earned (2 transactions)
  refund: $75.00 earned (1 transactions)
  freelance: $500.00 earned (1 trans

In [31]:
# Interactive AI Financial Assistant
# Ask questions about your financial data and get AI-powered insights!

# You can change this question to ask anything about your finances
my_question = "Based on my spending data showing $450 on shopping and $222 on food, what budget adjustments would you recommend?"

print(f"🤖 Your Question: {my_question}")
print("Thinking... (this may take 10-30 seconds)")

# Get AI response
ai_response = ask_ai(my_question)

print(f"\n💡 AI Answer:")
print(ai_response)

print("\n" + "="*50)
print("💡 Try changing 'my_question' above and rerun this cell!")
print("Example questions:")
print("- 'How can I budget better based on my spending patterns?'")
print("- 'What percentage of income should I save?'")
print("- 'How do I track my expenses more effectively?'")
print("- 'What are red flags in spending I should watch for?'")
print("="*50)

🤖 Your Question: Based on my spending data showing $450 on shopping and $222 on food, what budget adjustments would you recommend?
Thinking... (this may take 10-30 seconds)

💡 AI Answer:
Based on your spending data for January 15th, 2025, where you spent $450 on shopping and $222 on food, it appears that no budget adjustments are necessary.

The analysis shows that your spending in both categories is within or even below the allocated amounts. You spent exactly what was budgeted for shopping ($450) and less than what was allocated for food ($222). As a result, there's no need to adjust your budget for either category.

However, it's worth noting that if you had spent more than $450 on shopping or $222 on food, the query would have calculated the necessary adjustments based on the thresholds of 10% (1.2 multiplier) and 9% (1.1 multiplier), respectively. But since your spending is within these limits, no changes are required to your budget for January 15th, 2025.

💡 Try changing 'my_ques